In [2]:
RANDOM_SEED = 8924
rng = np.random.default_rng(RANDOM_SEED)
az.style.use("arviz-darkgrid")

In [3]:
bysubj_rev_df = pd.read_csv("bysubj_rev_df.csv")

In [4]:
# Assuming bysubj_rev_df is a pandas DataFrame with the necessary data
happiness = bysubj_rev_df["happiness"].values
epoch = bysubj_rev_df["epoch"].values
t = bysubj_rev_df["t"].values
user_ids = pd.Categorical(bysubj_rev_df["user_id"]).codes

n_users = len(np.unique(user_ids))
n_obs = len(happiness)

In [12]:
import pymc as pm
import pytensor.tensor as pt

with pm.Model() as model:
    # Priors for fixed effects
    intercept = pm.Normal("intercept", mu=0, sigma=1)
    beta_epoch = pm.Normal("beta_epoch", mu=0, sigma=1)
    beta_t = pm.Normal("beta_t", mu=0, sigma=1)
    beta_epoch_t = pm.Normal("beta_epoch_t", mu=0, sigma=1)

    # Non-centered reparameterization for the random effects
    offset = pm.Normal("offset", mu=0, sigma=1, shape=(n_users, 2))

    # LKJ Cholesky Covariance Priors
    chol, corr, stds = pm.LKJCholeskyCov(
        "chol_cov", n=2, eta=1, sd_dist=pm.HalfCauchy.dist(beta=2), compute_corr=True
    )

    # Apply the Cholesky factor to the offsets to get the random effects
    random_effects = pm.Deterministic("random_effects", pt.dot(chol, offset.T).T)

    # Extract random intercepts and slopes
    intercepts = random_effects[:, 0][user_ids]
    slopes = random_effects[:, 1][user_ids]

    # Model error
    sigma = pm.HalfCauchy("sigma", beta=2)

    # Expected value
    mu = (
        intercept
        + intercepts
        + (beta_epoch + slopes) * epoch
        + beta_t * t
        + beta_epoch_t * epoch * t
    )

    # Likelihood (Student-T distribution)
    y = pm.StudentT("y", nu=1, mu=mu, sigma=sigma, observed=happiness)

    # Sampling using JAX as the backend for improved performance
    trace = pm.sampling_jax.sample_numpyro_nuts(2000, chains=4, target_accept=0.99)

Compiling...
Compilation time = 0:00:13.684710
Sampling...
Compiling.. :   0%|          | 0/3000 [00:00<?, ?it/s]





Running chain 0:   0%|          | 0/3000 [00:04<?, ?it/s]




Running chain 0:  10%|█         | 300/3000 [01:54<14:49,  3.04it/s]

Running chain 0:  40%|████      | 1200/3000 [03:15<03:09,  9.48it/s]


Running chain 0:  65%|██████▌   | 1950/3000 [04:21<01:35, 10.97it/s]

Running chain 0:  75%|███████▌  | 2250/3000 [04:48<01:07, 11.13it/s]


Running chain 0:  85%|████████▌ | 2550/3000 [05:17<00:41, 10.82it/s]

Running chain 0:  90%|█████████ | 2700/3000 [05:31<00:27, 10.78it/s]

Running chain 0:  95%|█████████▌| 2850/3000 [05:47<00:14, 10.24it/s]

Running chain 0: 100%|██████████| 3000/3000 [06:21<00:00,  7.32it/s]















































Running chain 3: 100%|██████████| 3000/3000 [11:21<00:00,  4.40it/s]
Sampling time = 0:11:23.688557
Transforming variables...
Transformation time = 0:00:00.920579


In [14]:
az.summary(trace, round_to=2)

/Users/corrado/opt/anaconda3/envs/pymc9_env/lib/python3.11/site-packages/arviz/stats/diagnostics.py:592: RuntimeWarning: invalid value encountered in scalar divide
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
intercept,0.06,0.02,0.03,0.09,0.00,0.0,175.95,358.75,1.01
beta_epoch,-0.06,0.02,-0.10,-0.03,0.00,0.0,106.10,244.85,1.03
beta_t,0.02,0.00,0.01,0.03,0.00,0.0,4568.01,5924.87,1.00
beta_epoch_t,-0.01,0.01,-0.02,0.00,0.00,0.0,5768.13,5786.72,1.00
"offset[0, 0]",0.95,0.11,0.75,1.16,0.01,0.0,299.08,884.98,1.02
...,...,...,...,...,...,...,...,...,...
"random_effects[212, 1]",-0.02,0.07,-0.16,0.10,0.00,0.0,1290.34,3287.30,1.00
"random_effects[213, 0]",-0.25,0.02,-0.29,-0.21,0.00,0.0,352.54,1035.05,1.01
"random_effects[213, 1]",0.03,0.03,-0.02,0.09,0.00,0.0,239.05,1031.32,1.01
"random_effects[214, 0]",0.44,0.05,0.34,0.53,0.00,0.0,1464.06,2844.34,1.00
